# BERT (Bidirectional Encoder Representations from Transformers)

<div class="alert alert-block alert-danger">

For Practice Notebook 205, please download the notebook and run it on [Google Colab](https://colab.research.google.com/), as it requires GPU access. As of August 28, 2025, our GPU infrastructure is still under development. We will notify you once GPU-enabled containers become available.

</div>

We began with traditional word embeddings like Word2Vec (CBOW/Skip-gram). Now, let's examine **BERT**—Google's transformer-based model that has fundamentally changed how we represent text by generating **contextualized embeddings**. Unlike static embeddings, which assign a single vector to each word, BERT creates a unique representation for each word based on its context within a sentence.

## Comparing Word2Vec and BERT

| Feature | Word2Vec/GloVe | **BERT** |
| :-- | :-- | :-- |
| **Representation** | Static: one vector per word | Contextual: vector changes with context |
| **Context Handling** | Ignores context | Fully bidirectional (uses all context) |
| **Architecture** | Shallow neural network | Deep transformer with self-attention |
| **Tokenization** | Word-level | Subword-level (WordPiece) |
| **Out-of-Vocabulary** | Cannot handle unseen words well | Can compose subwords for unseen words |
| **Computation** | Fast, lightweight | Computationally intensive |
| **Typical Use Cases** | Quick similarity, basic tasks | Complex tasks: NER, QA, sentiment, etc. |
| **Example** | "bank" always same vector | "bank" in "river bank" vs "bank account" |

### What does this mean in practice?

- **Word2Vec**: The word "Missouri" always has the same vector, whether it appears in "University of Missouri" or "Missouri River."
- **BERT**: The vector for "Missouri" changes depending on whether it's used in an academic, geographic, or sports context.


## How BERT Works (Key Implementation Details)

1. **Tokenization**
    - BERT uses WordPiece tokenization, splitting rare words into subwords (e.g., "Columbia" → `columb`, `##ia` if not in vocabulary).
2. **Special Tokens**
    - `[CLS]` at the start (used for classification tasks)
    - `[SEP]` between sentences
3. **Output**
    - Produces a 768-dimensional vector for each token (in `bert-base-uncased`)
    - Utilizes 12 transformer layers, each providing contextual representations

## Why This Matters for Our Missouri-Focused NLP Work

When we analyze a sentence like:

> "The University of Missouri-Columbia researchers studied Missouri river ecosystems."

- **Word2Vec** gives the same embedding for "Missouri" in both "University of Missouri" and "Missouri river."
- **BERT** generates different embeddings:
    - "Missouri" in "University of Missouri" (academic context)
    - "Missouri" in "Missouri river" (geographical context)
    - "Columbia" in "Missouri-Columbia" (campus) vs. "Columbia" as a city

This **contextual awareness** enables us to:

- Distinguish between the university and the state in entity recognition
- Analyze sentiment about the University of Missouri-Columbia versus the state as a whole
- Classify regional topics with greater accuracy


## Practical Considerations

- **Word2Vec** is fast and efficient, suitable for simple tasks or when computational resources are limited.
- **BERT** is more powerful for nuanced understanding but requires more computation and is best for tasks where context matters (e.g., distinguishing between "Columbia" as a city vs. a university campus).

We will implement these concepts in our next notebook section, using Missouri-centric examples to show how BERT's contextual embeddings provide richer, more accurate text representations than traditional static embeddings.

# Example

## 1. Setup and Initialization

In [ ]:
# !pip install transformers  # Only run if needed

from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

**Why we do this:**
- We use `bert-base-uncased` because it's a standard starting point (case-insensitive, general vocabulary)
- The tokenizer handles WordPiece tokenization automatically
- The model provides 12 transformer layers with 768-dimensional outputs

## 2. Helper Functions

In [2]:
import numpy as np

def cosine_similarity(a, b):
    # Compute cosine similarity between two vectors a and b
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_bert_for_token(string, term):
    # Tokenize the input string using the BERT tokenizer
    inputs = tokenizer(string, return_tensors="pt")
    tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    try:
        # Find the index of the target token (term) in the tokenized list
        term_idx = tokens.index(term)
    except ValueError:
        # Raise an error if the token is not found in the sentence
        raise ValueError(f"Token '{term}' not found in: {tokens}")
    # Pass the tokenized input through the BERT model
    outputs = model(**inputs)
    # Extract and return the embedding for the specified token
    return outputs.last_hidden_state[0][term_idx].detach().numpy()

**Key design choices:**
1. **Cosine similarity**: Measures angular distance between vectors (better than Euclidean for embeddings)
2. **Token-matching approach**: 
   - Forces exact token match (BERT tokenizes "Columbia" as `columbia` but "Columbia" might be `columb` + `##ia`)
   - Uses first occurrence only (simplifies demonstration)
3. **Error handling**: Skips sentences where term isn't found exactly

## 3. Missouri-Focused Dataset

In [3]:
query = "The University of Missouri-Columbia is located in Missouri."
comp_sents = ["Missouri is known for its beautiful rivers and forests.",         # State context
              "I attended the University of Missouri-Columbia...",               # Academic context
              "The Missouri Tigers are... University of Missouri.",              # Sports context
              "Columbia is a vibrant college town in Missouri.",                 # Geographic context
              "The University of Missouri-Columbia has... journalism program.",  # Program context
              "Kansas City is... largest cities in Missouri."                    # Urban context
              ]

We deliberately choose sentences where "Missouri" refers to different things: the state, the university, sports teams, and geographic locations. This allows us to observe how BERT adapts the meaning of "Missouri" based on its context.

## 4. Contextual Comparison


We use a helper function to extract the embedding for the word "Missouri" from each sentence using BERT. For each sentence, we:

- Tokenize the sentence (using WordPiece tokenization)
- Find the position(s) of the token "missouri"
- Extract the corresponding vector from BERT's output

In [4]:
# Get the BERT embedding for the token "missouri" in the query sentence
query_rep = get_bert_for_token(query, "missouri")

vals = []
# For each comparison sentence, compute the cosine similarity of "missouri" embeddings
for sent in comp_sents:
    try:
        # Get the BERT embedding for "missouri" in the current sentence
        comp_rep = get_bert_for_token(sent, "missouri")
        # Compute cosine similarity between query and comparison embedding
        cos_sim = cosine_similarity(query_rep, comp_rep)
        # Store the similarity and sentences for later sorting/printing
        vals.append((cos_sim, query, sent))
    except ValueError:
        # Skip sentences where "missouri" token is not found
        continue

# Sort results by similarity (highest first), then print them
for c, q, s in reversed(sorted(vals)):
    print(f"{c:.3f}\t{q}\t{s}")

0.940	The University of Missouri-Columbia is located in Missouri.	The University of Missouri-Columbia has... journalism program.
0.902	The University of Missouri-Columbia is located in Missouri.	I attended the University of Missouri-Columbia...
0.784	The University of Missouri-Columbia is located in Missouri.	Columbia is a vibrant college town in Missouri.
0.757	The University of Missouri-Columbia is located in Missouri.	The Missouri Tigers are... University of Missouri.
0.698	The University of Missouri-Columbia is located in Missouri.	Kansas City is... largest cities in Missouri.
0.553	The University of Missouri-Columbia is located in Missouri.	Missouri is known for its beautiful rivers and forests.


**How to interpret these results:**

- **Highest similarity (0.940, 0.902):**
Sentences where "Missouri" is used in an academic/university context are most similar to the query. BERT recognizes that "Missouri" in "University of Missouri-Columbia" is closely related to "Missouri" in other university-related sentences.
- **Moderate similarity (0.784, 0.757):**
Sentences referencing "Missouri" in a geographic or institutional context (like "Columbia" as a town or "Missouri Tigers" as sports teams) are somewhat similar, but less so than the academic context.
- **Lowest similarity (0.698, 0.553):**
Sentences where "Missouri" clearly refers to the state as a geographic or demographic entity (rivers, cities) are least similar to the university context. BERT captures this distinction, assigning a more different embedding.


## Why BERT is Better Than Word2Vec Here

- **Word2Vec** would give the *same* vector for "Missouri" in all these sentences, so cosine similarity would always be high, regardless of context.
- **BERT** adapts the embedding of "Missouri" to each context, allowing us to distinguish when "Missouri" refers to the university, the state, or something else.


## Key Takeaways for Students

- **Contextual embeddings** let us distinguish between different meanings of the same word, which is crucial for tasks like entity recognition, sentiment analysis, and topic classification.
- BERT's approach is especially valuable for real-world text, where words often have multiple meanings depending on context—just like "Missouri" in our university and state examples.

**Next Steps:**
We encourage you to try this approach with other tokens, such as "Columbia" or "University," and see how their meanings shift across different sentences. You can also extend this analysis to full sentence embeddings or experiment with visualizing similarities using dimensionality reduction techniques.